In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import json
import pandas as pd

# Configuración de ChromeDriver utilizando ChromeDriverManager
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Descomenta esta línea si deseas usar modo headless
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Lista para almacenar los productos extraídos
productos = []

# Función para extraer el precio final desde las etiquetas HTML
def extraer_precio_final(product_element):
    try:
        precio_final_parts = product_element.find_elements(By.CSS_SELECTOR, "span.vtex-product-price-1-x-sellingPrice.vtex-product-price-1-x-sellingPrice--summary")
        precio_final = ''.join([part.text for part in precio_final_parts])
        if not precio_final:
            print("No se encontró el precio final.")
            return None
        print(f"Precio Final: {precio_final}")
        return precio_final
    except Exception as e:
        print(f"Error al extraer el precio final: {e}")
        return None

# Función para extraer el precio original desde las etiquetas HTML
def extraer_precio_original(product_element):
    try:
        precio_original_parts = product_element.find_elements(By.CSS_SELECTOR, "span.vtex-product-price-1-x-listPriceValue.vtex-product-price-1-x-listPriceValue--summary.strike")
        precio_original = ''.join([part.text for part in precio_original_parts])
        if not precio_original:
            print("No se encontró el precio original.")
            return None
        print(f"Precio Original: {precio_original}")
        return precio_original
    except Exception as e:
        print(f"Error al extraer el precio original: {e}")
        return None

# Función para extraer los productos desde el JSON-LD
def extraer_productos(url):
    driver.get(url)
    time.sleep(5)

    try:
        script_json = driver.find_element(By.XPATH, '//script[@type="application/ld+json"]').get_attribute('innerHTML')
        print(f"JSON extraído: {script_json[:500]}...")

        datos_json = json.loads(script_json)
        if '@type' in datos_json and datos_json['@type'] == 'ItemList':
            product_elements = driver.find_elements(By.CLASS_NAME, 'vtex-product-summary-2-x-container')
            
            for item, product_element in zip(datos_json['itemListElement'], product_elements):
                producto = item['item']
                nombre = producto.get('name', '')
                
                marca = ''
                if 'brand' in producto and '@type' in producto['brand'] and producto['brand']['@type'] == 'Brand':
                    marca = producto['brand'].get('name', '')

                color = producto.get('color', '')
                sku = producto.get('sku', '')

                precio_final = extraer_precio_final(product_element)
                precio_original = extraer_precio_original(product_element)

                imagen_url = producto.get('image', '')
                enlace_producto = producto.get('@id', '')

                producto_data = {
                    'Nombre': nombre,
                    'Marca': marca,
                    'Color': color,
                    'SKU': sku,
                    'Precio Final': precio_final,
                    'Precio Original': precio_original,
                    'URL de la Imagen': imagen_url,
                    'URL Página': enlace_producto,
                    'Fecha': '',
                    # Resto de campos con valor por defecto
                }

                productos.append(producto_data)

            # Guardar los productos en un DataFrame y exportar a Excel
            df_productos = pd.DataFrame(productos)
            output_path = r'C:\Users\constanza.perez\OneDrive - Colgram\Escritorio\KILLAH4.xlsx'
            df_productos.to_excel(output_path, index=False)
            print(f"Productos guardados.")
        else:
            print(f"JSON no contiene la estructura esperada para ItemList.")
    except Exception as e:
        print(f"Error al extraer los productos: {e}")

# Páginas a scrapear
total_paginas = 60

for i in range(1, total_paginas + 1):
    url = f'https://www.colloky.cl/accesorios?page={i}'
    extraer_productos(url)
    print(f"Página {i} procesada")

# Cerrar el navegador
driver.quit()


JSON extraído: {"@context":"https://schema.org","@type":"ItemList","itemListElement":[{"@type":"ListItem","position":1,"item":{"@context":"https://schema.org/","@type":"Product","@id":"https://www.colloky.cl/gorro-coleccion-nina-rosado-gola1635i25/p","name":"Gorro Tejido con Líneas Rosado","brand":{"@type":"Brand","name":"Colloky"},"image":"https://collokycl.vtexassets.com/arquivos/ids/1531201/GOLA1635I25_1.png?v=638736108726300000","description":"Gorro de lana ideal para el invierno","mpn":"GOLA1635MEI25","sk...
Precio Final: $9.990
No se encontró el precio original.
Precio Final: $9.990
No se encontró el precio original.
Precio Final: $14.990
No se encontró el precio original.
Precio Final: $9.990
No se encontró el precio original.
Precio Final: $9.990
No se encontró el precio original.
Precio Final: $10.990
No se encontró el precio original.
Precio Final: $10.990
No se encontró el precio original.
Precio Final: $10.990
No se encontró el precio original.
Precio Final: $39.990
No se e

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=132.0.6834.160)
Stacktrace:
	GetHandleVerifier [0x008C74A3+25091]
	(No symbol) [0x0084DC04]
	(No symbol) [0x0072B373]
	(No symbol) [0x0070CEDB]
	(No symbol) [0x00797FAF]
	(No symbol) [0x007AADC9]
	(No symbol) [0x00791CA6]
	(No symbol) [0x007631D5]
	(No symbol) [0x0076435D]
	GetHandleVerifier [0x00BC07C3+3142947]
	GetHandleVerifier [0x00BD1A2B+3213195]
	GetHandleVerifier [0x00BCC412+3191154]
	GetHandleVerifier [0x00968720+685184]
	(No symbol) [0x00856E1D]
	(No symbol) [0x00853E18]
	(No symbol) [0x00853FB6]
	(No symbol) [0x008466F0]
	BaseThreadInitThunk [0x75B05D49+25]
	RtlInitializeExceptionChain [0x77A9CEBB+107]
	RtlGetAppContainerNamedObjectPath [0x77A9CE41+561]
